In [23]:
import pandas as pd
import torch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

import warnings

warnings.simplefilter(action='ignore', category=DeprecationWarning)


In [2]:
torch.cuda.is_available()

True

In [3]:
organized_result = pd.read_csv('temp_result/organized_result_1.csv')
organized_result

,document,target,reason,fact,metadata
0,34829.pdf,行政院原住民族委員會及花蓮縣政府,花蓮區漁會辦理「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機採購案」，未能確實...,據立法委員函轉民眾陳訴行政院原住民族委員會(下稱原民會)補助花蓮區漁會給二予漁民架設衛星導航...,"{'kw': ""['政府', '行政院', '案件', '原住民', '計畫', '漁探機'..."
1,34838.pdf,行政院衛生署,為行政院衛生署辦理九十至九十三年度補、捐助計畫，經該署檢討至少計一二五項計畫應以委辦而竟以補...,本案係立法院鄭委員三元向本院陳訴略以「行政院衛生署將應以委辦執行之經費，改由補助費方式辦理，...,"{'kw': ""['計畫', '捐助', '年度', '行政院', '案件', '意旨', ..."
2,36946.pdf,台北縣萬里鄉公所、台北市政府、基隆市政府、台北縣政府,台北市政府、基隆市政府及台北縣政府未善盡土地使用管制、環境污染稽查及申報資料勾稽、查核之責，...,本案緣陳訴人檢附採證照片陳訴台北市政府核准設立之新生代環境清潔公司（下二稱新生代公司）、統徠...,"{'kw': ""['政府', '公司', '清除', '垃圾', '顯有', '核准', '..."
3,36997.pdf,行政院國軍退除役官兵輔導委員會台北榮民總醫院,行政院國軍退除役官兵輔導委員會台北榮民總醫院辦理「醫學科技大樓法定停車場工程」興建發包作業，...,本案前經審計部函報行政院國軍退除役官兵輔導委員會（以下簡稱退輔會）台北榮民總醫院（以下簡稱台...,"{'kw': ""['工程', '提升', '於九', '法定', '完成', '台北', '..."
4,37008.pdf,行政院國軍退除役官兵輔導委員會,為行政院國軍退除役官兵輔導委員會為改善所屬十一所榮民醫院及十八所安養機構之污水處理廠及衛生下...,行政院國軍退除役官兵輔導委員會﹙以下簡稱退輔會﹚為改善所屬十一所榮民醫院及十八所安養機構之廢...,"{'kw': ""['工程', '政府', '下水道', '方式', '核有', '污水', ..."
...,...,...,...,...,...
71,62812.pdf,雲林縣褒忠鄉公所,雲林縣褒忠鄉公所辦理111年度「褒忠鄉大手牽小手為愛一起走」活動，由東衛傳播有限公司（下稱東...,本案緣於雲林縣褒忠鄉鄉長陳建名等人，疑利用職務上機會詐取財物及從事不當行為，涉犯貪污治罪條例...,"{'kw': ""['依法', '年度', '褒忠', '利益', '公司', '東衛', '..."
72,62931.pdf,空軍司令部、海軍司令部,空軍司令部辦理「PIT輪盤等25項採購」及「一級輪盤等8項」等採購案，得標廠商繳交生產日期及...,審計部派員調查空軍司令部及海軍司令部辦理「噴嘴等118項」等18件軍機或軍艦引擎料件採購案，...,"{'kw': ""['司令部', '禁止', '顯有', '提升', '安全', '引擎', ..."
73,63324.pdf,新竹縣新豐鄉公所,新竹縣新豐鄉公所對於清潔隊人員出缺之甄補作業未能審慎辦理，致予機關首長有藉由該項人事決定權收...,地方鄉、鎮公所首長濫用清潔隊員人事進用及核定之權限，要求、期約、收受賄賂之案例層出不窮，屏東...,"{'kw': ""['公所', '政府', '決標', '方式', '藉由', '致予', '..."
74,63770.pdf,台灣電力股份有限公司(下稱台電公司),台電公司審議立式車床採購規格，以年代久遠無相關採購資料為由，僅制定最大加工工件尺寸及載重為規...,台電公司電力修護處洪姓課長(下稱洪員)，涉以「限制廠商資格」、「制定特殊產品規格」等方式，於...,"{'kw': ""['政府', '訪價', '未能', '下稱', '損及', '有失', '..."


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)

In [5]:
document_reason_loader = DataFrameLoader(organized_result, page_content_column='reason')
document_reason = document_reason_loader.load()

document_target_loader = DataFrameLoader(organized_result, page_content_column='target')
document_target = document_target_loader.load()

In [26]:
# embed_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embed_model_id = "sentence-transformers/distiluse-base-multilingual-cased-v1"

core_embedding_model = HuggingFaceBgeEmbeddings(model_name=embed_model_id)

store = LocalFileStore("./cache/")
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embedding_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(document_reason, embedder)
# vector_store = FAISS.from_documents(document_target, embedder)

e:\miniconda3\envs\project_envelop\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
query = """行政院"""
embedding_vector = core_embedding_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k=5)

for page in docs:
    print(page.page_content)

行政院國家科學委員會
行政院原子能委員會
行政院農業委員會
行政院衛生署
行政院原住民族委員會及花蓮縣政府


In [8]:
model_id = "DavidLanz/Llama2-tw-7B-v2.0.1-chat"
device_map = {"": 0}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4', # normalized float 4
    bnb_4bit_compute_dtype='float16', # compute in float point 16
    bnb_4bit_use_double_quant=True # double quant to compress the model
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device_map,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_auth_token=True
)

model.eval()

e:\miniconda3\envs\project_envelop\lib\site-packages\transformers\models\auto\auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.91s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [11]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token = '',
)

In [18]:
generate_text = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=1.0,
    max_length=2048,
    max_new_tokens=256
)

In [19]:
llm = HuggingFacePipeline(pipeline=generate_text)

In [20]:
cb = StreamingStdOutCallbackHandler()
llm("### USER: 請告訴我2024年1月台積電的股價狀況? \n### ASSISTANT: ", callbacks=[cb])

Both `max_new_tokens` (=256) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
e:\miniconda3\envs\project_envelop\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


'### USER: 請告訴我2024年1月台積電的股價狀況? \n### ASSISTANT: 我無法預測未來的股票價格，也無法提供未來的市場分析和趨勢預測。股價的波動和變化因素有很多，沒有任何可靠的方法可以確定特定時間的股票價格。 此外，市場波動是常見的，我們也不能保證股票價格將保持穩定。'

In [31]:
template = """你是監察院資料查詢服務機器人。

{context}

只使用提供的來源資料回答客戶的問題。如果您不確定，請說："我不知道，請致電給我們的客戶支援"。使用禮貌專業的語言。
保持回答簡潔摘要，並列出資料來源出處。

問題：

答案："""

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context"])
print(
    prompt.format(
        context = "一位客戶希望透過資料查詢AI服務機器人提供與他問題相關的資料。"
    )
)

In [27]:
# top_k = 3
# query = "國防部的採購案有哪些"
# embedding_vector = core_embedding_model.embed_query(query)
# docs = vector_store.similarity_search_by_vector(embedding_vector, k=top_k)

# reference = []
# for page in docs:
#     reference.append(page.page_content)
#     # print(page.page_content)

# from langchain.prompts import PromptTemplate

# template = """
# ### INSTRUCTION: 請根據 REF 回答 USER 的問題
# ### REF: {reference}
# ### USER: {query}
# ### ASSISTANT: """
# prompt = PromptTemplate.from_template(template)

# chain = prompt | llm

# data = {
#     "reference": " ".join(reference),
#     "query": query,
# }
# # print(data)
# result = chain.invoke(data)


Both `max_new_tokens` (=256) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [30]:
reference

['國防部所屬海軍司令部、空軍司令部、空軍官校、航技學校及生製中心等採購單位辦理「103年度教準部鎮海、嘉華營區營舍整修工程」等19件採購案，不同投標廠商之投標文件有重大異常關聯等情，嚴重影響採購公帄性，惟機關審標人員均未能發現及時處理，致政府採購法規定及工程會相關解釋令（函）形同具文經審計部密送國防部（總督察長室）查明，國防部卻僅依廠商回函說明，未督促所屬釐清是否涉及不法，並檢討相關人員責任；且國防部總督察長室職掌該部軍風紀，本應藉其客觀超然之督察地位，統整發揮機關內控功能，秘密對涉嫌不法之採購案，進行財務稽核及軍紀監察；卻逕將審計部密函轉交國防採購室辦理；國防採購室又層層下轉海軍司令部、空軍司令部、軍備局及其下屬採購單位，完全失去審計部密送總督察長之原意，均有違失，爰依法提案糾正',
 '國防部辦理空軍第二後勤指揮部軍工廠委託民間經營採購案，對於陳情廠商106年9月26日陳情函，未落實依採購稽核小組組織準則規定妥處；另辦理本案過程與政府採購法第51條及採購評選委員會審議規則等相關規定容屬有間；又未於事前掌握評選委員出席人數，並及時因應更改評選委員會議時間，以獲得更多委員出席參與評選；且本案經辦過程洵有未盡周延之處，造成廠商多次質疑及申訴，影響政府施政效能，難辭監督不周疏失責任，國防部及所屬空軍司令部均有違失，爰依法提案糾正',
 '國防部軍備局辦理「海軍測校磁站裝備整修」勞務採購案，招標文件規定全案12項工程均為主要部分，依政府採購法規定應由得標廠商自行履行，不得由其他廠商代為履行，否則即違反不得轉包之規定查得標廠商○○工程股份有限公司提出之整修企劃書內容說明其協力合作廠商，且提出之「台電蘭嶼核廢料與放射性廢料桶檢整重建計畫處理中心作業系統之建立與運轉案」實績證明，並未含括全案12項工程主要部分，明顯違反規定，國防部軍備局不察仍審查合格，顯有違失，爰依法提案糾正']

In [29]:
print(result)


### INSTRUCTION: 請根據 REF 回答 USER 的問題
### REF: 國防部所屬海軍司令部、空軍司令部、空軍官校、航技學校及生製中心等採購單位辦理「103年度教準部鎮海、嘉華營區營舍整修工程」等19件採購案，不同投標廠商之投標文件有重大異常關聯等情，嚴重影響採購公帄性，惟機關審標人員均未能發現及時處理，致政府採購法規定及工程會相關解釋令（函）形同具文經審計部密送國防部（總督察長室）查明，國防部卻僅依廠商回函說明，未督促所屬釐清是否涉及不法，並檢討相關人員責任；且國防部總督察長室職掌該部軍風紀，本應藉其客觀超然之督察地位，統整發揮機關內控功能，秘密對涉嫌不法之採購案，進行財務稽核及軍紀監察；卻逕將審計部密函轉交國防採購室辦理；國防採購室又層層下轉海軍司令部、空軍司令部、軍備局及其下屬採購單位，完全失去審計部密送總督察長之原意，均有違失，爰依法提案糾正 國防部辦理空軍第二後勤指揮部軍工廠委託民間經營採購案，對於陳情廠商106年9月26日陳情函，未落實依採購稽核小組組織準則規定妥處；另辦理本案過程與政府採購法第51條及採購評選委員會審議規則等相關規定容屬有間；又未於事前掌握評選委員出席人數，並及時因應更改評選委員會議時間，以獲得更多委員出席參與評選；且本案經辦過程洵有未盡周延之處，造成廠商多次質疑及申訴，影響政府施政效能，難辭監督不周疏失責任，國防部及所屬空軍司令部均有違失，爰依法提案糾正 國防部軍備局辦理「海軍測校磁站裝備整修」勞務採購案，招標文件規定全案12項工程均為主要部分，依政府採購法規定應由得標廠商自行履行，不得由其他廠商代為履行，否則即違反不得轉包之規定查得標廠商○○工程股份有限公司提出之整修企劃書內容說明其協力合作廠商，且提出之「台電蘭嶼核廢料與放射性廢料桶檢整重建計畫處理中心作業系統之建立與運轉案」實績證明，並未含括全案12項工程主要部分，明顯違反規定，國防部軍備局不察仍審查合格，顯有違失，爰依法提案糾正
### USER: 國防部的採購案有哪些
### ASSISTANT: 國防部的採購案有哪些？
國防部所屬海軍司令部、空軍司令部、空軍官校、航技學校及生製中心等採購單位辦理「103年度教準部鎮海、嘉華營區營舍整修工程」等19件採購案，不同投標廠商之投標文件有重大異常關聯等情，嚴重影響採購公帄性，但機關審標人員均未能發現及時處理，致

In [25]:
# template = """
# ### INSTRUCTION: 請根據 REF 回答 USER 的問題
# ### REF: {reference}
# ### USER: {query}
# ### ASSISTANT: """
# prompt = PromptTemplate.from_template(template)

# chain_type_kwargs = {"prompt": prompt}

# chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1}, max_tokens_limit=250, return_source_documents=False),
#     # retriever=vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.3, "k": 1}, max_tokens_limit=250, return_source_documents=False),
#     chain_type_kwargs=chain_type_kwargs,
# )